# Black hole rendering with SageMath

**Written by Tae Geun Kim from *Florentin Jaffredo***

In [1]:
version()

'SageMath version 9.3, Release Date: 2021-05-09'

In [2]:
%display latex

## Table of Contents

* [Configuration](#Configuration)
* [Declaring the Spacetime](#Declaring-the-spacetime)
* [Launching a Geodesic](#Launching-a-Geodesic)

## Configuration

In [3]:
n_cpu = 16
n_geod = 1000
nx, ny = 720, 360

## Declaring the Spacetime

### Manifold Structure

In [4]:
M = Manifold(4, 'M', structure='Lorentzian')

In [5]:
M

4-dimensional Lorentzian manifold M

In [6]:
C.<t, r, th, ph> = M.chart(r't r:(1,+oo) th:(0,pi):\theta ph:\phi')

In [7]:
C

Chart (M, (t, r, th, ph))

In [8]:
C.coord_range()

t: (-oo, +oo); r: (1, +oo); th: (0, pi); ph: (-oo, +oo)

### Metric Structure

We use Schwarzschild metric

In [9]:
m = var('m', domain='positive')

In [10]:
g = M.metric()
g[0,0] = -(1 - 2*m/r)
g[0,1] = 2*m/r
g[1,1] = 1 + 2*m/r
g[2,2] = r^2
g[3,3] = (r*sin(th))^2

In [11]:
g

Lorentzian metric g on the 4-dimensional Lorentzian manifold M

In [12]:
g[:]

[    2*m/r - 1         2*m/r             0             0]
[        2*m/r     2*m/r + 1             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

In [13]:
g.display()

g = (2*m/r - 1) dt*dt + 2*m/r dt*dr + 2*m/r dr*dt + (2*m/r + 1) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

### Locally Euclidean Space

In [14]:
E.<x,y,z> = EuclideanSpace()
phi = M.diff_map(E, [r*sin(th)*cos(ph), r*sin(th)*sin(ph), r*cos(th)])
phi.display()

M --> E^3
   (t, r, th, ph) |--> (x, y, z) = (r*cos(ph)*sin(th), r*sin(ph)*sin(th), r*cos(th))

## Launching a Geodesic 

### Define a point

In [15]:
p = M((0, 14.98, pi/2, 0))
p

Point on the 4-dimensional Lorentzian manifold M

### Define a tangent space

In [16]:
Tp = M.tangent_space(p)
Tp

Tangent space at Point on the 4-dimensional Lorentzian manifold M

### Define a tangent vector

In [17]:
v = Tp((2, 0, 0.005, 0.05))
v = v / sqrt(-g.at(p)(v, v))
v

Tangent vector at Point on the 4-dimensional Lorentzian manifold M

In [18]:
v.display()

2/sqrt(-0.534045393858478*m + 3.43338899) d/dt + 0.00500000000000000/sqrt(-0.534045393858478*m + 3.43338899) d/dth + 0.0500000000000000/sqrt(-0.534045393858478*m + 3.43338899) d/dph

In [19]:
tau = var('tau')
curve = M.integrated_geodesic(g, (tau, 0, 3000), v)

In [20]:
sol = curve.solve(step = 1, method="ode_int", parameters_values={m: 2})

In [21]:
interp = curve.interpolate()

In [22]:
P = curve.plot_integrated(mapping=phi, color='red', thickness=2, plot_points=3000)
P += sage.plot.plot3d.shapes.Sphere(4, color='grey')
P

Graphics3d Object

## Launching a lot of Geodesics

In [23]:
import multiprocessing
from ipywidgets import FloatProgress
from IPython.display import display

In [24]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [25]:
n_batches_per_cpu = 3

In [26]:
curve = M.integrated_geodesic(g, (tau, 0, 200), v, across_charts=True)

In [27]:
args = []
start_index = 0

for chunk in chunks(range(n_geod), n_geod//(n_batches_per_cpu*n_cpu)):
    args += [(loads(curve.dumps()), start_index, len(chunk))]
    start_index += len(chunk)

In [28]:
print(args[-1])
print(len(args))

(Integrated geodesic in the 4-dimensional Lorentzian manifold M, 980, 20)
50


In [29]:
dt, y, r0 = var('dt, y, r0')

In [30]:
p = M((0, r0, pi/2, 0))
Tp = M.tangent_space(p)
v = Tp((dt, -1, 0, y))
v.display()

dt d/dt - d/dr + y d/dph

In [31]:
g.at(p)(v, v)

r0^2*y^2 + dt^2*(2*m - r0)/r0 - 4*dt*m/r0 + (2*m + r0)/r0

In [32]:
sol = g.at(p)(v, v).solve(dt)
sol

[dt == -(sqrt(-(2*m*r0 - r0^2)*y^2 + 1)*r0 - 2*m)/(2*m - r0), dt == (sqrt(-(2*m*r0 - r0^2)*y^2 + 1)*r0 + 2*m)/(2*m - r0)]

In [33]:
def calc_some_geodesics(args):
    curve, n0, nb = args
    res = {}
    r = 100
    posi = [0, r, pi/2, 0]
    p = M(posi)
    Tp = M.tangent_space(p)
    for i in range(n0, n0+nb):
        dy = i * 0.006/n_geod
        v = Tp([sol[0].rhs()(r0=r, y=dy, m=2).n(), -1, 0, dy])
        # overwrite the starting vector
        curve._initial_tangent_vector = v
        # integration with m=2
        curve.solve_across_charts(step=0.2, parameters_values={m:2})
        # copy and clear solution
        res[i] = (p.coord(), curve._solutions.copy())
        curve._solutions.clear()
    return res

In [34]:
geo = {}
pool = multiprocessing.Pool(n_cpu)

# progress bar display
#%display plain
f = FloatProgress(min=0, max=n_geod)
display(f)

for i, some_res in enumerate(pool.imap_unordered(calc_some_geodesics, args)): # do and wait
    # progress bar update
    f.value += len(some_res)
    # update result
    geo.update(some_res)

# clean exit
pool.close()
pool.join()

FloatProgress(value=0.0, max=1000.0)

In [35]:
# add the sphere
P = sage.plot.plot3d.shapes.Sphere(4, color='grey')

# cycle through the solutions
for i in range(0, n_geod, 5*n_geod/100):    
    # set solution
    curve._solutions = geo[i][1]
    # do interpolation
    interp = curve.interpolate()
    # plot the curve
    P += curve.plot_integrated(mapping=phi, color=["red"], thickness=2, plot_points=150, 
                               label_axes=False, across_charts=True)

# show the result    
P

Graphics3d Object

### Intersection with the accretion disk

In [36]:
disk_min = 12
disk_max = 50
alpha = -pi/20

In [37]:
D = sage.plot.plot3d.shapes.Torus((disk_min + disk_max)/2,
                                 (disk_min-disk_max)/2).scale(1,1,0.01).rotateY(-pi/20)

In [38]:
D

Graphics3d Object

In [39]:
P + D

Graphics3d Object

In [40]:
P + D.rotateX(pi/3)

Graphics3d Object